# Additional tools for PB-M3C2

<p style="color:red;"> <a style="font-weight: bold">
WARNING:</a> The implementation of this method is experimental and under active development.
</p>

In this notebook, we will provide extension to the [PB-M3C2 workflow](pbm3c2.ipynb) that will be occasionally useful based on your application.

## Generation of non-correspondent pairs

For best training results, the user should provide both pairs of segments that do correspond to each other, as well as pairs of segments that do not correspond. In manual labelling workflows, it is much easier to produce high quality corresponding pairs that it is to produce non-corresponding pairs. Here, we provide a function that allows you to generate pairs of non-corresponding segments automatically based on heuristic. The general procedure is exactly the same as in [the base workflow](pbm3c2.ipynb) and will not be further explained here.

In [ ]:
import py4dgeo
import pooch

In [ ]:
py4dgeo.set_interactive_backend("vtk")

In [ ]:
# Set up pooch to download data from Zenodo
p = pooch.Pooch(base_url="doi:10.5281/zenodo.16751963/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()

try:
    # Download and extract the dataset
    p.fetch("usage_data.zip", processor=pooch.Unzip(members=["usage_data"]))

    # Define path to the extracted data
    data_path = p.path / "usage_data.zip.unzip" / "usage_data"
    print(f"Data path: {data_path}")

    # Read XYZ files from the extracted directory
    epoch0, epoch1 = py4dgeo.read_from_xyz(
        data_path / "plane_horizontal_t1.xyz", data_path / "plane_horizontal_t2.xyz"
    )

except Exception as e:
    print(f"Failed to download or extract data: {e}")

In [ ]:
alg = py4dgeo.PBM3C2()

In [ ]:
(
    xyz_epoch0,
    xyz_epoch1,
    extracted_segments,
) = alg.export_segmented_point_cloud_and_segments(
    epoch0=epoch0,
    epoch1=epoch1,
)

Now, we will use labelling data from the file `testdata-labelling-correspondent-only.csv`, which does not contain any pairs of non-corresponding segments. Running `add_no_corresponding_seg` on this data, we automatically generate these. There are two heuristics that can be selected through the `algorithm` parameter:
* `random`: For each segment in one epoch, label a random segment from the neighborhood in the other epoch as non-corresponding.
* `closes`: For each segment in one epoch, take the closest segment in the other epoch and label it non-corresponding.

The neighborhood of a segment is defined by the threshold parameter given as `threshold_max_distance`.

In [ ]:
augmented_extended_y = py4dgeo.add_no_corresponding_seg(
    segments=extracted_segments,
    threshold_max_distance=5,
    algorithm="random",
    extended_y_file_name=str(data_path / "testdata-labelling-correspondent-only.csv"),
)

We can then run the training algorithm, passing directly the augmented labelling data:

In [ ]:
alg.training(
    extracted_segments_file_name="extracted_segments.seg",
    extended_y=augmented_extended_y,
)

In [ ]:
distances, uncertainties = alg.compute_distances(epoch0=epoch0, epoch1=epoch1)